In [5]:
import json
import numpy as np
import pandas as pd
#import geopandas as gpd
import json
import re
import os

from tqdm.notebook import tqdm
from opencage.geocoder import OpenCageGeocode

# ============================================
# clean tranaction price df
# ============================================
def clean_transaction_price():
    # load raw data:
    df = pd.read_csv('../../data/raw/transaction_price.csv', lines=True,orient='records').drop_duplicates()
    df = df.loc[~df.district.isnull()]
    print(f'Orignal df transaction price: {df.shape}')
    
    # clean district name:
    df['district'] = df['district'].map(lambda x: re.sub('/', '-',str(x)) )
    df['district'] = df['district'].map(lambda x: re.sub(',', '-',str(x)) )
    df['district'] = df['district'].map(lambda x: re.sub('\s', '',str(x)) )
    df['district'] = df['district'].str.title()
    
    # number of rooms
    df['apartment_type'] = df['apartment_type'].str.lower()
    df['number_of_room'] = df['apartment_type'].map(lambda x: x[:1] if not pd.isna(x) else None)
    df['number_of_room'] = pd.to_numeric(df['number_of_room'] , errors='coerce')
    
    df['number_of_room_cat'] = df['number_of_room'].map(lambda x: str(round(x)) if not pd.isna(x) else None)
    df['number_of_room_cat'] = df['number_of_room'].map(lambda x: '3+' if float(x) >=3 else x)
    df['number_of_room_cat'] = df['number_of_room_cat'].astype('str')
    
    # property type
    df['property_type']= df['property_type'].replace({'kt':'apartment', 'rt':'row house', \
                                                  'ok':'single-family house'})
    df['property_type_main'] = df['property_type'].map(lambda x: 'house' if x!='apartment' else x)
    
    return df
    
def geocode_address(df):
    key = os.getenv('opencage_api')
    geocoder = OpenCageGeocode(key)
    postcode_dict = {}
    district_geocode = df['district'].unique() # find list of unique postcode
    district_geocode = [s + ', Finland'  for s in district_geocode]

    for i in range(0, len(district_geocode)):
        result= geocoder.geocode(district_geocode[i])
        if len(result) > 1:            
            postcode_dict[district_geocode[i]]= result[1]['components']['postcode']
        else:       
            postcode_dict[district_geocode[i]]= pd.DataFrame(result[0])['formatted'].iloc[0]

            
    return postcode_dict    

def merge_district(postcode_dict, df):
    df_postcode = pd.DataFrame(postcode_dict, index=[0]).T.reset_index()      
    df_postcode.columns= ['district', 'postcode']
    
    df_postcode['district'] = df_postcode['district'].str.replace(', Finland', '')
    df_postcode['postcode'] = df_postcode['postcode'].map(lambda s: re.findall('\d{5}', s) )
    df_postcode['postcode'] = df_postcode['postcode'].map(lambda x: x[0] if len(x) >0 else None)
                                                         
    df = df.merge(df_postcode, on='district')
            
    return  df

def geocode_address(df):
    key = os.getenv('OPENCAGE_API_KEY')
    geocoder = OpenCageGeocode(key)

    postcode_dict = {}
    postcode_null = []
    district_geocode = df['district'].unique() # find list of unique postcode
    district_geocode = [s + ', Finland'  for s in district_geocode]

    for i in range(0, len(district_geocode)):
        result= geocoder.geocode(district_geocode[i])
        if len(result) > 1:
            try:
                postcode_dict[district_geocode[i]]= result[1]['components']['postcode']
            except :
                print(district_geocode[i])
                postcode_null.append(district_geocode[i])
        else:       
            postcode_dict[district_geocode[i]]= pd.DataFrame(result[0])['formatted'].iloc[0]
      
    return postcode_dict, postcode_null    


def merge_district(postcode_dict, df):  
    '''Merge 2 gecode dataframe so we have postcodes in transaction price'''
    df_postcode = pd.DataFrame(postcode_dict, index=[0]).T.reset_index()   
    df_postcode.columns= ['district', 'postcode']
    
    df_postcode['district'] = df_postcode['district'].str.replace(', Finland', '')
    df_postcode['postcode'] = df_postcode['postcode'].map(lambda s: re.findall('\d{5}', s) )
    df_postcode['postcode'] = df_postcode['postcode'].map(lambda x: x[0] if len(x) >0 else None)
                                                          
    df= df.merge(df_postcode, on='district')
    print('Geocode postcode for df transaction price: done')
            
    return  df
 
def process_transaction_df():
    df = clean_transaction_price()
    postcode_geo, postcode_null = geocode_address(df) 
    df = merge_district(postcode_geo, df)
    
    df.to_csv('../../data/processed/transaction_price.csv', index=False)
    print('==== Save df transaction price: done ====')
       
    return df


# ============================================
# clean asking price df
# ============================================

def clean_df_listing():
    df = pd.read_csv("../data/raw/Espoo_Vantaa.csv").drop_duplicates()
    df['postcode'] = df['postcode'].astype(str).str.pad(5, "left", "0")
    df['asking_price_m2'] = df['price'] / df['floor_area']

    # clean municipality
    df["municipality"]=df["municipality"].astype('category')
    # number_of_room
    df['number_of_room'] = df['house_info'].str.extract(pat =  "(\d\s*m*h*\s?\+*)")
    df['number_of_room'] = df['number_of_room'].str.extract('(\d+)')
    df['number_of_room'] = df['number_of_room'].map(lambda x: float(x) if not pd.isna(x) else x)

    df['number_of_room_cat'] = df['number_of_room'].map(lambda x: str(round(x)) if not pd.isna(x) else None)
    df['number_of_room_cat'] = df['number_of_room'].map(lambda x: '3+' if float(x) >=3 else x)
    df['number_of_room_cat'] = df['number_of_room_cat'].astype('str')
    
    # maitenance cost per sqm
    df['maintenance_cost_m2'] = df['Unit maintenance cost'] / df['floor_area']
    df['maintenance_cost_m2'] = df['maintenance_cost_m2'].map(lambda x: 0 if pd.isna(x) else x)
    
    # filter potential wrong values
    df = df[ ~((df['number_of_room'] <1 )| (df['number_of_room'].isnull() )) ]
    df = df[(df['asking_price_m2']>1)  &  (df['asking_price_m2']<20000)] # local knowledge: rarely there is \
                                                           # property sold with >20k/m2
    # divide propery into 2 groups: house and apartment:
    df['property_type_main'] = df['property_type'].map(lambda x: 'house' if x!='apartment' else x)
    
    df.to_csv('../data/processed/asking_price.csv', index=False)
    print('==== Save df asking price: done ====')
    
    return df

# ============================================
# clean rent df
# ============================================

def load_rent():
    # load file
    df = pd.read_json('../data/raw/rent.jl', lines=True,orient='records').drop_duplicates()
    
    # clean columns
    df['postcode'] = df['postcode'].map(lambda x: re.sub('\D+', '', x))
    df['number_of_room'] = df['apartment_type'].map(lambda x: re.sub('h', '', x))
    df = df.loc[df['number_of_room'].isin(['1', '2', '3+'])]
    
    # change dtype of rent columns
    df[['nonsub_old', 'nonsub_new']] = df[['nonsub_old', 'nonsub_new']].apply(pd.to_numeric, errors='coerce')
    
    # drop unneeded columns:
    df.drop(['apartment_type', 'ARA_rental'], axis=1, inplace=True)

    df.to_csv('../data/processed/rent.csv', index=False)
    print('==== Save df rent: done ====')
    
    return df


# ============================================
# merge df_rent, df_price and df_listing
# ============================================

def caculate_yield(x):
    if x['new']:
        x['net_yield'] = 12*(x['nonsub_new'] - x['maintenance_cost_m2']) / x['asking_price_m2']
    else:
        x['net_yield']= 12*(x['nonsub_old'] - x['maintenance_cost_m2']) / x['asking_price_m2']
    
    x['net_yield']= np.round(x['net_yield'] * 100, 2)
    
    return x['net_yield']

def merge_df(df_rent, df_listing, df_price):
    # correct format of postcode:
    for df in [df_rent, df_listing, df_price]:
        df['postcode'] = df['postcode'].map(lambda x: str(np.int(x)) if not pd.isna(x) else x )
        df['postcode'] = df['postcode'].astype(str).str.pad(width=5, side='left', fillchar='0')
        if 'number_of_room_cat' in df.columns:
            df['number_of_room_cat'] = df['number_of_room_cat'].replace({'1.0': '1' , '2.0':'2'})
            
    # merge price and listing

    df_price = df_price.groupby(['postcode', 'number_of_room_cat','property_type_main'])\
                ['price_m2'].mean().reset_index()
    df_listing_agg = df_listing.groupby(['postcode', 'number_of_room_cat','property_type_main'])\
                ['asking_price_m2'].mean().reset_index()
    
    df_listing_price = df_listing_agg.merge(df_price[['postcode', 'number_of_room_cat','price_m2']], on=['postcode', 'number_of_room_cat'],\
                      how='left').drop_duplicates()
    
    # merge df listing and df rent
    df_yield = df_listing.merge(df_rent, left_on = ['postcode', 'number_of_room_cat'], \
                            right_on = ['postcode', 'number_of_room'], how='left')
   
    print(df_yield['maintenance_cost_m2'].isnull().mean())
    df_yield['new'] = df_yield['build_year'].map(lambda x: True if x >= 2015 else False)
    df_yield['net_yield'] = df_yield.apply(caculate_yield, axis=1)
    
    df_yield.to_csv('../data/processed/df_yield.csv', index=False)
    df_listing_price.to_csv('../data/processed/df_listing_price.csv', index=False)
   
    print('==== Save df yield and price: done ====')
    
    
    return df_listing_price, df_yield
        

if __name__ == "__main__":
    print('Start')
    # df_rent = load_rent()
    df_price = process_transaction_df()
    # df_listing = clean_df_listing()
    # df_listing_price, df_yield =  merge_df(df_rent, df_listing, df_price)


Start


TypeError: read_csv() got an unexpected keyword argument 'lines'

In [2]:
! pip install geopandas

  Using cached geopandas-0.9.0-py2.py3-none-any.whl (994 kB)
  Using cached pyproj-3.0.1.tar.gz (168 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\user\anaconda3\python.exe' 'C:\Users\user\anaconda3\lib\site-packages\pip\_vendor\pep517\_in_process.py' get_requires_for_build_wheel 'C:\Users\user\AppData\Local\Temp\tmpbryh56_f'
       cwd: C:\Users\user\AppData\Local\Temp\pip-install-k21uhy5z\pyproj
  Complete output (1 lines):
  proj executable not found. Please set the PROJ_DIR variable. For more information see: https://pyproj4.github.io/pyproj/stable/installation.html
  ----------------------------------------
ERROR: Command errored out with exit status 1: 'C:\Users\user\anaconda3\python.exe' 'C:\Users\user\anaconda3\lib\site-packages\pip\_vendor\pep517\_in_process.py' get_requires_for_build_wheel 'C:\Users\user\AppData\Local\Temp\tmpbryh56_f' Check the logs for full command output.
